# TODO
#### Klassen protecten
```
import AutoMLWrapper.automlwrapper as aw
aw.Configuration.Configuration
```

#### Erweiterung um Bibliotheken
-  @__library.setter
    - a) Global/GlobalConfig.yaml
        - lib-names
        - type mappings
    - b) directory durchsuchen 
- import in AutoMLWrapper anpassen
- SEDAR automl.py config locations anpassen

### custom proprocessing
- typen checken
- die wrapper könnten property methods implementieren, die die Rückgabe eines festen samples prüfen

### custom parameters


### 2+ problem types

In [1]:
import io
import mlflow
import pandas as pd
import numpy as np
from PIL import Image

from AutoMLWrapper.tests import DataSamples

#from automlwrapper import AutoMLWrapper
from AutoMLWrapper.automlwrapper import AutoMLWrapper



2024-01-07 20:34:19.083745: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 20:34:19.144055: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 20:34:19.145178: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 20:34:20.242791: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


In [2]:
RUN_ALL = 0
UPLOAD_ALL = 0

RUN_KERAS = 0
UPLOAD_KERAS = 0

RUN_SKLEARN = 0
UPLOAD_SKLEARN = 0

RUN_GLUON = 0
UPLOAD_GLUON = 0

user_hp = {'epochs':20, 'time_limit': 360, 'memory_limit':7000, 'testtest': 123, 'num_trials': 4,}

In [3]:
MLFLOW = 0

if MLFLOW:
    
    # #experiment_id = mlflow.create_experiment('test_automl', artifact_location='/home/mibbels/sedar-masterarbeit/sedar/sedar/automl/tmp/artifacts')
    
    experiment_id = 5
    remote_server_uri = "http://127.0.0.1:6798"
    mlflow.set_tracking_uri(remote_server_uri) 
    mlflow.set_experiment(experiment_id=experiment_id)

# Daten

In [4]:
""" Bilddaten """
#mnist_byte_df = DataSamples.create_mnist_bytearray_df(label_col='label', n_samples=-1)
shopee_path_df = DataSamples.create_shopee_df(is_bytearray = False, n_samples=-1)
#mnist_tp = DataSamples.create_mnist_tuple(n_samples=-1)
#leaf_df = DataSamples.create_leaf_df(n_samples=-1)

coco_df = DataSamples.create_coco_motorbike_df(n_samples=-1)

""" Textdaten """
#sentiment_df = DataSamples.create_sentiment_treebank_df(n_samples=-1)
mloc_df = DataSamples.create_mloc_df(n_samples=-1)

""" Tabellendaten """
glass_df = DataSamples.create_glass_df()

""" Timeseries """
m4_df = DataSamples.create_m4_df(n_samples=-1)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [5]:
coco_df

,image,rois,label
0,./data/tiny_motorbike_coco/tiny_motorbike/Anno...,"[[352.0, 138.0, 499.0, 374.0, 7], [105.0, 1.0,...","[[352.0, 138.0, 499.0, 374.0, 7], [105.0, 1.0,..."
1,./data/tiny_motorbike_coco/tiny_motorbike/Anno...,"[[40.0, 71.0, 332.0, 472.0, 7], [33.0, 42.0, 3...","[[40.0, 71.0, 332.0, 472.0, 7], [33.0, 42.0, 3..."
2,./data/tiny_motorbike_coco/tiny_motorbike/Anno...,"[[52.0, 22.0, 307.0, 327.0, 8], [26.0, 108.0, ...","[[52.0, 22.0, 307.0, 327.0, 8], [26.0, 108.0, ..."
3,./data/tiny_motorbike_coco/tiny_motorbike/Anno...,"[[114.0, 154.0, 368.0, 347.0, 7], [292.0, 49.0...","[[114.0, 154.0, 368.0, 347.0, 7], [292.0, 49.0..."
4,./data/tiny_motorbike_coco/tiny_motorbike/Anno...,"[[120.0, 333.0, 309.0, 463.0, 7]]","[[120.0, 333.0, 309.0, 463.0, 7]]"
...,...,...,...
164,./data/tiny_motorbike_coco/tiny_motorbike/Anno...,"[[1.0, 143.0, 111.0, 235.0, 7], [164.0, 114.0,...","[[1.0, 143.0, 111.0, 235.0, 7], [164.0, 114.0,..."
165,./data/tiny_motorbike_coco/tiny_motorbike/Anno...,"[[1.0, 187.0, 496.0, 332.0, 7], [124.0, 15.0, ...","[[1.0, 187.0, 496.0, 332.0, 7], [124.0, 15.0, ..."
166,./data/tiny_motorbike_coco/tiny_motorbike/Anno...,"[[205.0, 24.0, 379.0, 286.0, 8], [118.0, 82.0,...","[[205.0, 24.0, 379.0, 286.0, 8], [118.0, 82.0,..."
167,./data/tiny_motorbike_coco/tiny_motorbike/Anno...,"[[192.0, 48.0, 320.0, 365.0, 8], [39.0, 123.0,...","[[192.0, 48.0, 320.0, 365.0, 8], [39.0, 123.0,..."


In [6]:
ask = AutoMLWrapper('autosklearn')
ask.SetOutputDirectory('test/ask/2')

if RUN_ALL or RUN_SKLEARN:
    ask.Train(
        data=mnist_byte_df,
        target_column='Type',
        task_type='classification',
        data_type='tabular',
        problem_type='multiclass',
        hyperparameters=user_hp
    )


if UPLOAD_ALL or UPLOAD_SKLEARN:
    print(ask.Output())
    ask.MlflowUploadBest({})

In [7]:
agl = AutoMLWrapper('autogluon')
agl.SetOutputDirectory('test/agl/8')

if RUN_ALL or RUN_GLUON: 
    agl.Train(
        data=mnist_byte_df,
        target_column='label',
        data_type='image',
        task_type='zero-shot-classification',        
        problem_type='multiclass',
        hyperparameters=user_hp
   )
    
if UPLOAD_ALL or UPLOAD_GLUON:
    print(agl.Output())
    agl.MlflowUploadBest({})

In [8]:
akr = AutoMLWrapper('autokeras')
akr.SetOutputDirectory('test/akr/1')

if RUN_ALL or RUN_KERAS:
    akr.Train(data=df_mnist, target_column='Type',
        task_type='classification',
        data_type='image',
        problem_type='multiclass',
        hyperparameters=user_hp
    )

if UPLOAD_ALL or UPLOAD_KERAS:
    print(akr.Output())
    akr.MlflowUploadBest({})

# MLFlow
~/miniconda3/envs/automl_env/bin/pip

In [9]:
if MLFLOW:
    df = df_glass.drop(columns=['Type'])
    logged_model = 'runs:/a0337b9ecfe84e99b59ff3a9639f1ef6/model'
    loaded_model = mlflow.pyfunc.load_model(logged_model)

    #loaded_model.predict(test_data_byte)

# SEDAR API

2024-01-09 20:43:03.991473: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-09 20:43:04.051907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-09 20:43:04.053049: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-09 20:43:05.154675: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


# DataLoader TODO
- sedar api raw sourcedata
- sedar api get_all_images (image metadata)

- query_sourcedata nur structured
    - image
    - unstructured
    - semistructured

- keras format

# Ingestion
- metadaten?

In [3]:
from sedarapi import SedarAPI
from AutoMLWrapper.automlwrapper.SedarDataLoader import SedarDataLoader


2024-01-11 13:13:48.026940: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-11 13:13:48.087889: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-11 13:13:48.088905: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-11 13:13:49.138901: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


In [4]:
base_url = "http://192.168.220.107:5000"
email = "admin"
password = "admin"

sedar = SedarAPI(base_url)
sedar.connection.logger.setLevel("INFO")
sedar.login(email, password)

INFO:SedarAPI-Logger:Login successful


In [5]:
workspace_id = '13b4787c3e454649aa05a4cd680edc37'
dataset_id = '324ea420125d4167a76151b62368c4ad'

# b5b74391e41e4634a54d5cffa059663b - coco_22_train_v1 
# 324ea420125d4167a76151b62368c4ad - gitter_train_v1
# 513c6b1ee46b478c8e0925a098d2f387  - test classification 1 img each class

from AutoMLWrapper.automlwrapper.SedarDataLoader import SedarDataLoader

DataLoader = SedarDataLoader(sedar)
loc = DataLoader.query_data(workspace_id, dataset_id, file_save_location='./data/sedar_raw/test')

INFO: the data has been written to: ./data/sedar_raw/test/coco_22_balanced.zip


In [6]:
DataLoader.zip_to_coco_df_gluon(loc[0], './data/sedar_raw/test/test_coco')

,image,label
0,./data/sedar_raw/test/test_coco/coco_22_balanc...,"[[212.99967999999998, 227.00006399999998, 218...."
1,./data/sedar_raw/test/test_coco/coco_22_balanc...,"[[385.999872, 322.00012799999996, 199.999488, ..."
2,./data/sedar_raw/test/test_coco/coco_22_balanc...,"[[408.0, 366.999936, 140.99968, 132.999936], [..."
3,./data/sedar_raw/test/test_coco/coco_22_balanc...,"[[378.999808, 360.00038399999994, 174.000128, ..."
4,./data/sedar_raw/test/test_coco/coco_22_balanc...,"[[466.00038399999994, 312.000384, 140.000256, ..."
...,...,...
65,./data/sedar_raw/test/test_coco/coco_22_balanc...,"[[433.000448, 314.00025600000004, 234.999808, ..."
66,./data/sedar_raw/test/test_coco/coco_22_balanc...,"[[416.999424, 246.000384, 220.000256, 216.9999..."
67,./data/sedar_raw/test/test_coco/coco_22_balanc...,"[[351.000064, 219.999744, 236.000256, 241.9998..."
68,./data/sedar_raw/test/test_coco/coco_22_balanc...,"[[426.00038399999994, 294.99993600000005, 181...."


In [19]:




if 0:
    ws = sedar.get_workspace('13b4787c3e454649aa05a4cd680edc37')
    dataset = ws.get_dataset('')

    
    try:
        success = dataset.delete()
        if success:
            print("Dataset deleted successfully.")
    except Exception as e:
        print(e)

INFO:SedarAPI-Logger:The Dataset '394a58ddfd564a2badc83797e786cab6' was deleted successfully.


Dataset deleted successfully.
